In [ ]:
import torch
import numpy as np
import matplotlib
from scipy import ndimage
import os, sys
import math
import pickle
import model_utils as modutil
import data_utils as datutil
import hmc

In [ ]:
# Data loader initialization
trainloader1 = datutil.generate_dataloaders('CIFAR10_TRAIN', batch_size=50, shuffle=True, num_workers=2)
testloader1 = datutil.generate_dataloaders('CIFAR10_TEST', batch_size=10, shuffle=False, num_workers=2)
validloader1 = datutil.generate_dataloaders('CIFAR100_TEST', batch_size=10, shuffle=False, num_workers=2)

trainloader2 = datutil.generate_dataloaders('ENCODED256_D110_CIFAR10_TRAIN', batch_size=50, shuffle=True, num_workers=2)
testloader2 = datutil.generate_dataloaders('ENCODED256_D110_CIFAR10_TEST', batch_size=10, shuffle=False, num_workers=2)
validloader2 = datutil.generate_dataloaders('ENCODED256_D110_CIFAR10_VALID', batch_size=10, shuffle=False, num_workers=2)

# Cifar-100 interesting classes (used during out-of-class entropy calculation)
interesting_labels = [0, 1, 16, 17, 20, 21, 29, 39, 40, 49, 57, 71, 72, 73, 76]

In [ ]:
# model to train/load/analyse
# user defined params
models = [{'model_type' : "PreResNet110",     # <Kernel_name> + <GP>
    'saved_checkpoint_name' : "PreResNet110_depth-110_lr-0.1_mom-0.9_wd-0.0001_FC-10_acc-91.07-2019-06-18-19.52",
    'fc_setup' : [],
    'load_model' : True,
    'train_model' : False,
    'train_epoch' : 20,
    'num_classes' : 10,
    'weight_decay' : 3e-4,
    'predef_test_acc' : 90,
    'depth' : 110,
    'grid_size' : 64,
    'gp_kernel_feature' : 256, # 256, 640
    'print_init_model_state' : False},]

In [ ]:
for model in models:
    
    if 'encoded' not in model['model_type']:
        trainloader = trainloader1
        testloader = testloader1
        validloader = validloader1
    else:
        trainloader = trainloader2
        testloader = testloader2
        validloader = validloader2
        
    print('='*20, "Loading Model", '='*20,)
    modutil.refresh_params()
    for propt in model:
        if propt in modutil.__dict__:
            modutil.__dict__[propt] = model[propt]
        else:
            print("Model property '%s' not found!"%(propt))
    
    #load / train the model
    modutil.load_train(trainloader, testloader)
    if model['train_model']:
        print("Saving model!")
        modutil.save_model()

    # param_list = param_chain if 'mcmc' in model['model_type'] else []
    # Perform evaluation on model
    # modutil.validate("out-of-class", validloader, interesting_labels=interesting_labels)
    print("Validation (out of class) data analysis performed")

    # modutil.validate("test", testloader)
    print("Test data analysis performed")


In [ ]:
test_file = 'saved_predictions/' + models[0]['saved_checkpoint_name'] + '.testpred'
prob_cutoffs = [(i+1)/30.0 for i in range(30)]

with open(test_file, 'rb') as test_file:
    stats = pickle.load(test_file)
    targets = stats['targets']
    preds = stats['predictions']
    probs = stats['probs']
    ece, ece_avg = modutil.calculate_ECE(probs, preds, targets, ECE_bin=prob_cutoffs)

print(ece, ece_avg)